In [1]:
from N3_FL import *
from json import loads

In [2]:
# Add bogus payment model
class Transaction:
    def __init__ (self, chain, sender, recipient, load, type_, payment): #type_ - mod/pay/glb
        self.sender = sender
        self.load = load
        self.recipient = recipient
        self.type = type_
        self.payment = payment
        self.chain = chain
    
    def __str__(self):
        r = "TRX: C{0:5.5}:{1:5.5}-->{2:5.5} | {3:5.5} | ¤{4:4.2f} | Load: ".format(str(self.chain), str(self.sender), str(self.recipient), self.type, self.payment) + self.load
        return r

In [3]:
class Block:
    def __init__(self, chain, chain_idx, trxs, proof, prev_hash, ts=None, repu_list=None):
        self.chain = chain
        self.chain_idx = chain_idx
        self.transactions = trxs
        self.proof = proof
        self.prev_hash = prev_hash
        if ts == None:
            self.timestamp = time()
        else:
            self.timestamp = ts
        if repu_list == None:
            self.reputation_list = {}
        else:
            self.reputation_list = repu_list

    def __str__(self):
        r = "BLK: C:{0:7.7}#{1:3d} <-- PH={2:5.5}:PF={3:5.5}, RL={4:5.5}, #TRXS={5}".format(str(self.chain.bc_id),self.chain_idx,str(self.prev_hash),str(self.proof),str(self.reputation_list), str(len(self.transactions)))
        return r

In [4]:
def removeCli(txns, cli): #in dict
    tx_new = []
    for t in txns:
        if t.type == 'PAY':
            tx_new.append(t)
        if t.type == 'MOD':
            if t.sender == cli:
                pass
            else:
                tx_new.append(t)
    return tx_new

def truncateViewRL(rl):
    r = ""
    for c, q in rl.items():
        r += "{0:5.5}:{1:.4}; ".format(c,q)
    return r

In [5]:
# MAKE CONVERTER TO OTHER ROLE VIA TRX DATA?????
class Node(Client):
    def __init__(self, id_):
        super().__init__(id_)
        self.blockchain = None
        self.balance = 0
        self.committee = True
        self.mine_benchmarks = []

    def initSet(self):
        if hasattr(self, 'blockchain') == False:
            self.blockchain = None
            self.balance = 0
    
    def __str__(self):
        return "{0}:{3}-{1:5.5}, {2:7d}".format(self.getType(), \
                            self.hash, self.getVolume(), self.id_)

    def getType(self):
        return "NDE"

    def assignBlockchain(self, blockchain):
        self.blockchain = blockchain
        self.blockchain.authorizeClient(self)

    def importChain(self, blockchain): # chain: type Blockchain
        self.blockchain = deepcopy(blockchain)
        last_trxs = blockchain.lastBlock.transactions
        last_gb = None
        for t in last_trxs:
            if t.type == 'GLB':
                last_gb = t.load
                break
        print(last_gb)
        #self.replaceWeights(global)
                        
    def evaluate(self):
        qual = self.model.model.evaluate(self.test_inp, self.test_oup, verbose=0)
        # assign to a client property\
        return qual

    def volAsHash(self):
        loc_vol = './temp/t1.txt'
        vol_hash = self.ipfs.sendToIPFS(saveToFile(str(self.getVolume()/self.roundLimit), loc_vol))
        rm(loc_vol)
        return vol_hash

    def submitModelAsTransaction(self):
        # send model HASH and MULTIPLIER (number of data)
        load = "{}|{}|{}|{}".format(self.hash, self.hashes[0], self.hashes[1], self.volAsHash()) # mod, qual, vol
        trx = Transaction(self.blockchain.bc_id, self.id_, 0, load, 'MOD', 0)
        chidx = self.blockchain.newTransaction(trx)
        print("{0}:{1} MOD-->CHIDX#{2:3d}".format(self.getType(),str(self.id_), chidx))

    def evaluateMine(self, model_weights):
        model_ = CNN()
        model_.setup()
        model_.replaceModel(model_weights)
        qual = model_.model.evaluate(self.test_inp, self.test_oup, verbose=2)
        # assign to a client property\
        return qual[1]
    
# COMMITTEE FUNCTIONS_________________________________________________________________________________
    def aggregate(self, blockchain):
        if self.committee == False:
            return
    
        # Initiate resulting global model
        global_model = None
        ###################### MOVE TO BC
        if blockchain.proofOfAuthority(c.id_) == False:
            print("Unauthorized")
            return
        
        loc = './models/'
        self.mine_benchmarks = []
        # IMPORT PAST QUALITIES
        # REPU LIST: QUAL (VOL * ACC)
        past_qualities = blockchain.lastBlock.reputation_list

        for t in blockchain.current_transactions:
            # parse t
            cli_hash, mod_hash, qual_hash, vol_hash = t.load.split('|')
            model_ = IPFStoModObj(self.ipfs, mod_hash, loc)
            vol_ = int(float(IPFStoObj(self.ipfs, vol_hash, loc)))
            qual_ = float(IPFStoObj(self.ipfs, qual_hash, loc))
            
            # Test everything against itself
            # qual_ = REPORTED qual = vol ** accu
            # compare to c.evalmine

            benchmarks = {'hash': cli_hash, 'qual_mined' : self.evaluateMine(model_) * (vol_ ** 0.5), \
                            'vol' : vol_, 'qual_reported' : qual_, 'model_file' : model_}
            self.mine_benchmarks.append(benchmarks)
            print(benchmarks['hash'][:5], benchmarks['qual_mined'], benchmarks['qual_reported'])

        # Quality check from previous versions of qualities
        # calculate past qualities average first
        past_average_qual = 0
        for record in past_qualities:
            past_average_qual += record['qual'] / sum(past_qualities)

        print(past_qualities)

        improvement_threshold = 0.7
        average_threshold = 0.3
        bm_pass = []
        for bm in self.mine_benchmarks:
            if len(past_qualities) > 0:
                for record in past_qualities: # THIS ONE WONT RUN IF EMPTY
                    if bm['hash'] in record['hash']:
                        # LOC_NOW vs LOC_PAST
                        if float(bm['qual_mined']) > improvement_threshold * float(record['qual']):
                            bm_pass.append(bm)
                            continue
                        # LOC_NOW vs GLOB_PAST
                        if float(bm['qual_mined']) > average_threshold * past_average_qual:
                            bm_pass.append(bm)
                    else:
                        bm_pass.append(bm)
            else:
                bm_pass.append(bm)
        self.mine_benchmarks = bm_pass

        sel_hashes = []
        for h in self.mine_benchmarks:
            sel_hashes.append(h['hash'])

        # Aggregate from benchmarks
        # formula: qual_report + qual_mined
        if len(self.mine_benchmarks) > 1:
            models_multiply = []
            models_multiply_proportion = []
            models = []    
            for bm in self.mine_benchmarks:
                # get weights and models
                models_multiply.append(bm['qual_mined'] + bm['qual_reported'])
                models.append(bm['model_file'])
            
            for m in models_multiply:
                models_multiply_proportion.append(m / sum(models_multiply))
            
            models_to_combine = []
            for i in range(len(models)):
                models_to_combine.append(multiply_scalar(models[i].get_weights(), models_multiply_proportion[i]))

            # aggregate
            global_model = CNN()
            global_model.setup()
            #print(models_to_combine)
            global_model.model.set_weights(combine_models_multiple(models_to_combine))
        elif len(c.mine_benchmarks) == 1:
            global_model = CNN()
            global_model.setup()
            global_model.model.set_weights(self.mine_benchmarks[0]['model_file'].get_weights())
        else:
            #NO MINE
            print('Nothing to mine')
        return global_model

    def mine(self): # RETURN BLOCK
        print("Start mining: " + self._id)
        
        # GET TRANSACTIONS
        self.last_trxs = deepcopy(self.blockchain.current_transactions)

        t_hashes = [] #fill with (id, hash, hashM)
        for t in self.last_trxs:
            if t.type == 'MOD':
                t_hashes.append((t.sender,t.load,None))
        
        # PoA
        if self.blockchain.proofOfAuthority(self.id_) == False:
            return

        loc = "./models/"
        t_creds = []
        # Obtain quality
        for t in t_hashes:
            t_mod, t_amt = t[1].split('|')
            t_id = t[0]
            # download
            self.getFromIPFS(t_mod,loc)
            self.getFromIPFS(t_amt,loc)
            # open files
            fH = open(loc + t_amt, "r")
            mul = fH.read()
            fH.close()
            md = load_model(loc+t_mod).get_weights()
            t_creds.append((t_id, md, mul))
            rm(loc+t_mod)
            rm(loc+t_amt)

        bogus_model = CNN()
        bogus_model.model.compile(optimizer='adam',loss='mse')         
        
        rl_current = {}
        for t in range(len(t_creds)):
            t_id, t_mod, t_mul = t_creds[t][0], t_creds[t][1], int(t_creds[t][2])
            bogus_model.model.set_weights(t_mod)
            bogus_model.model.build((None,self.frame_in,1))
            bogus_model.model.compile(optimizer='adam',loss='mse')         
            # prediction phase
            dev = self.predict(bogus_model, t_mod)
            # quality calculation
            ##OOOOOOOOOOOOOOOOOOOOOOOOOOO A D D : Q U A L   F R O M   E A C H   N O D E
            qual = (1/dev)[0] * (1 + log(t_mul, 1000)) * (1 + log(self.getMultiplier(), 1000))
            rl_current[t_id] = qual

        resp = ""
        for ci, qu in rl_current.items():
            resp += "{0}|{1};".format(ci,qu)

        # send qual to IPFS and attach to proposed block
        f = open(loc+'tempQ','w')
        f.write(resp)
        f.close()
        hashQ = self.ipfs.sendToIPFS(loc+'tempQ')
        rm(loc+'tempQ')
        # PoQ
        self.blockchain.proofOfQuality(self.id_, hashQ)
        return

In [6]:
class Compounder(Node):
    def __init__(self, id_):
        super().__init__(id_)
        self.ocNodes = []
        self.model_eval = CNN()
        self.model_eval.setup()
    
    def initSet(self):
        if hasattr(self, 'ocNodes') == False:
            self.ocNodes = []
            self.model_eval = CNN()
            self.model_eval.setup()

    def getType(self):
        return "CPD"

    def registerOCNode(self, node):
        self.ocNodes.append(node)

    def examine(self):
        m_evals = []
        m_lens = []
        m_mods = []
        for c in self.ocNodes:
            self.model_eval.replaceModel(c.model)
            mo = c.model.model.get_weights()
            le = c.getVolume()
            ev = self.model_eval.model.evaluate(self.test_inp, self.test_oup, verbose=2)
            m_evals.append(ev)
            m_lens.append(le)
            m_mods.append(mo)
        m_evals.append(self.model.evaluate(self.test_inp, self.test_oup, verbose=2))
        m_lens.append(self.getVolume())
        m_mods.append(self.model.model.get_weights())
        return m_mods, m_lens, m_evals

    def train(self, loc, epoch_, round_):
        self.model.model.set_weights(self.combineModels())
        return super().train(loc, epoch_, round_)

    def combineModels(self):
        c_mods, c_lens, c_evals = self.examine()
        c_contribs = []
        c_props = []
        
        for c in range(len(c_evals)):
            c.contribs.append(c_evals[c] * (c_lens[c] ** 0.5))

        for c in range(len(c_evals)):
            c_props.append(c_contribs[c]/sum(c_contribs))
        
        mod_combined = multiply_scalar(c_mods, c_props)
        mod_combined = combine_models_multiple(mod_combined)
        return mod_combined

In [7]:
class OffChainNode(Client):
    def __init__(self, id_):
        super().__init__(id_)
        self.id_ = toHash(id_)

    def initSet(self):
        pass
    
    def getType(self):
        return "OCN"

    def __str__(self):
        return "{0}:{3}-{1:5.5}, {2:7d}".format(self.getType(), \
                            self.hash, self.getVolume(), self.id_)
    
    def requestBind(self, cli):
        cli.registerOCNode(self)

In [8]:
# define which nodes are off-chain first
#def authorize(blockchain, clients):
#    for c in clients:
#        c.assignBlockchain(blockchain)

# THINK ON HOW TO USE THE OFFCHAIN NODES ETC
#def train(clients, epochs_):
#    for c in clients:
#        c.train()
#server = Server(clients, epochs_, rounds_)
#server.setTestCase(test_inp_, test_oup_)
#return server

In [9]:
def setupClisF(train_inps_, train_oups_, test_inps_, test_oups_, rounds_, client_poi, classes_initial):
    clients = []
    for i in range(len(train_inps_)):
        c = classes_initial[i](i)
        c.model.setup()
        c.setData(train_inps_[i], train_oups_[i], test_inps_[i], test_oups_[i])
        c.setRoundLimit(rounds_)
        if i in client_poi:
            c = poisonFL(c)
        clients.append(c)
    return clients
    
def bindOCNodes(binds, clients):
    # binds: [cliOC, cliComp]
    for b in binds:
        clients[b[0]].requestBind(clients[b[1]])

In [10]:
model_dir = "./models/"
#count = 7
count = 2
test_ratio = 0.25
epochs = 2
rounds = 2
client_poisoning = [[], [5,4],[6,4,3],[1],[4,2,1],[0],[2,0]]
ep_it = [(2,1),(4,1),(8,1),(16,1),(1,2),(1,4),(2,2),(4,2),(2,4),(4,4),(8,2),(2,8)]

try:
    train_inps = pickle.load(open("./temp/trinps", "rb"))
    train_oups = pickle.load(open("./temp/troups", "rb"))
    test_inps = pickle.load(open("./temp/teinps", "rb"))
    test_oups = pickle.load(open("./temp/teoups", "rb"))
    test_inp = pickle.load(open("./temp/teinp", "rb"))
    test_oup = pickle.load(open("./temp/teoup", "rb"))
except:
    train_inp, test_inp, train_oup, test_oup = split(fetch(), test_ratio)
    train_inps = buildPortions(count, train_inp)
    train_oups = buildPortions(count, train_oup)
    test_inps = buildPortions(count, test_inp)
    test_oups = buildPortions(count, test_oup)
    pickle.dump(train_inps, open("./temp/trinps", "wb"))
    pickle.dump(train_oups, open("./temp/troups", "wb"))
    pickle.dump(test_inps, open("./temp/teinps", "wb"))
    pickle.dump(test_oups, open("./temp/teoups", "wb"))
    pickle.dump(test_inp, open("./temp/teinp", "wb"))
    pickle.dump(test_oup, open("./temp/teoup", "wb"))

In [11]:
def saveModelH(model, ipfs):
    save_loc = './models/glob0'
    model.model.save(save_loc, save_format='h5')
    hash = ipfs.sendToIPFS(save_loc)
    rm(save_loc)
    return hash

def saveFileToLoc(content, loc):
    f = open(loc, 'w')
    f.write(content)
    f.close()
    return loc

In [12]:
class Blockchain(object): # PROVING NODE: ID + SALT + HASH
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        self.bc_id = str(hex(id(self)))
        self.block_backlog = []
        self.clients = set()
        self.cli_pointers = []
        self.newBlock(previous_hash=0, proofA=0, proofQ=0, trxs=[]) # CHANGE PREV HASH AND PROOF ???
        self.salt = str(randint(1,100000)) # salt for this blockchain
        print("Blockchain created with ID: " + self.bc_id + " " + str(int(self.bc_id,base=16)))
        self.ipfs = IPFS(self.bc_id)
        self.latest_repu_inc = {}
        self.mineGenesisBlock()

    def authorizeClient(self, client):
        # hash is secret to the
        cid_hash = toHash(client.id_ + self.salt)
        self.clients.add(cid_hash)
        self.cli_pointers.append(client)

    def proofOfAuthority(self, clientId):
        if toHash(clientId + self.salt) not in self.clients:
            print("{0:5.5} MINE REJECTED: UNAUTH".format(clientId))
            return False
        print("{0:5.5} MINE APPROVED: POA".format(clientId))
        return True

    def newTransaction(self, trx):
        self.current_transactions.append(trx)
        return self.lastBlock.chain_idx + 1

    @property
    def lastBlock(self):
        return self.chain[-1]
    
    def mineGenesisBlock(self):
        self.chain.append(Block(self, 0, [], 0, 0, 0, None))

    def newBlock(self, proofA, proofQ, trxs, previous_hash=None, mode="MINE", incentive_list=None): # int str |> dict
        if previous_hash == None:
            previous_hash = toHash(self.lastBlock)
        try:
            block = Block(self,self.lastBlock.chain_idx + 1,
                            trxs, proofA, 
                            previous_hash,
                            repu_list=proofQ)
        except Exception as e:
            print(e, end='|')
            print("Brand new blockchain, creating genesis block")
            block = Block(self,0,[], proofA, proofQ, repu_list={})
            self.chain.append(block)

        if mode == "MINE":
            print(block)
            # ADD TO BACKLOG
            self.block_backlog.append(block)
        elif mode == "CONSENSUS":
            self.incentivize(incentive_list)
            for i in self.current_transactions:
                if i.type == "PAY":
                    trxs.append(i)
            for tx in trxs:
                if tx.type == "PAY":
                    for ci in self.cli_pointers:
                        if tx.sender == ci.id:
                            ci.balance -= tx.payment
                        if tx.recipient == ci.id:
                            ci.balance += tx.payment
                    print(tx)
            if len(trxs) > 0:
                self.chain.append(block)
        return block

    def incentivize(self, inc):
        # SEPARATE TRX AND MODEL TRX
        l1_inc = inc[0]
        l2_inc = inc[1]
        for c, i in l1_inc.items():
            for cl in self.cli_pointers:
                if cl.id == c and i != 0:
                    tx = Transaction(self.bc_id,0,cl.id,'L1 Incentivization',"PAY",i)
                    self.newTransaction(tx)

        # MOVE TO MINER SIDE???
        for min, cli in l2_inc.items():
            for c, amt in cli.items():
                multiplier = 0.30
                if min == c:
                    continue
                tx = Transaction(self.bc_id,min,c,'L2 Incentivization', "PAY", amt * multiplier)
                self.newTransaction(tx)

    def consensus(self):
        loc = "./models/"
        cl_rl = {}
        for blk in self.block_backlog:
            cl, rl = blk.proof, blk.reputation_list
            try:
                self.getFromIPFS(rl,loc)
                f = open(loc + rl, 'r')
                rle = f.read()
                f.close()
                rm(loc+rl)

                rl = {}
                rl_clis = rle.split(';')[:-1]
                for l in rl_clis:
                    k = l.split('|')
                    rl[k[0]] = float(k[1])

                cl_rl[cl] = rl
            except Exception as e:
                pass
                #print(e)
        
        res = {}

        # Get max
        for k, r in cl_rl.items():
            if len(res) == 0:
                for k, v in r.items():
                    res[k] = v
            else:
                for k, v in r.items():
                    if r[k] > res[k]:
                        res[k] = r[k]

        rl_prev = self.lastBlock.reputation_list

        if type(rl_prev) == str:
            self.getFromIPFS(rl_prev, loc)
            f = open(loc + rl_prev, 'r')
            rl_pre = f.read()
            f.close()
            rm(loc + rl_prev)
            rl_prev = loads(rl_pre.replace("\'","\""))

        c_inc = {}
        inc_l2 = {}

        # Give credit
        for cli, rli in cl_rl.items():
            c_inc[cli] = 0
            inc_l2[cli] = {}
            for k, v in rli.items():
                if rli[k] == res[k]:
                    try:
                        diff = rli[k] - rl_prev[k]
                        if diff > 0:
                            c_inc[cli] += diff
                    except KeyError:
                        if c_inc[cli] != 0:
                            c_inc[cli] += rli[k]
                        else:
                            c_inc[cli] = rli[k]
                    try:
                        diff = rli[k] - rl_prev[k]
                        if diff > 0:                        
                            inc_l2[cli][k] = rli[k] - rl_prev[k]
                    except:
                        inc_l2[cli][k] = rli[k]

        proofA = ""
        for cli in c_inc.keys():
            if c_inc[cli] > 0:
                proofA += str(cli) + "|"
        
        rl_latest = {}
        for k, cr in cl_rl.items():
            for c, r in cr.items():
                rl_latest[c] = r

        loc_ = "./models/tempD"
        f = open(loc_, "w")
        f.write(str(rl_latest))
        f.close()
        proofQ = self.ipfs.sendToIPFS(loc_)
        rm(loc_)

        txs_filter = []
        for cli, rl in inc_l2.items():
            for r in rl.keys():
                txs_filter.append(r)
        
        txs_selected = []

        for t in self.current_transactions:
            if t.type == 'PAY':
                txs_selected.append(t)
            if t.sender in txs_filter:
                txs_selected.append(t)
                
        block = self.newBlock(proofA, proofQ, txs_selected, mode="CONSENSUS", incentive_list=[c_inc,inc_l2])
        print(block, " -- CONSENSUS")
        self.current_transactions = []

        return c_inc

    def proofOfQuality(self, clientId, hashQ):
        # get prev stuff and see diffs
        rl_prev = self.lastBlock.reputation_list

        loc = './models/'

        # load hashQ
        self.getFromIPFS(hashQ, loc)
        f = open(loc + hashQ, 'r')
        rl_current = f.read()
        f.close()
        rm(loc + hashQ)

        if type(rl_prev) == str:
            self.getFromIPFS(rl_prev, loc)
            f = open(loc + rl_prev, 'r')
            rl_pre = f.read()
            f.close()
            rm(loc + rl_prev)
     
            rl_prev = loads(rl_pre.replace("\'","\""))

        dc_rl = {}
        rl_clis = rl_current.split(';')[:-1]
        for l in rl_clis:
            k = l.split('|')
            dc_rl[k[0]] = float(k[1])

        print("Proposal: ", truncateViewRL(dc_rl))
        trxs = deepcopy(self.current_transactions)

        updated = {}

        for cli, qual in dc_rl.items():
            if cli not in rl_prev.keys():
                updated[cli] = qual
            elif rl_prev[cli] < qual:
                updated[cli] = qual
            else:
                trxs = removeCli(trxs, cli)
        
        try:
            print("PREV = "+truncateViewRL(rl_prev)+ " | CURR = "+truncateViewRL(updated))
        except KeyError:
            print("PREV=\{\} | CURR="+truncateViewRL(updated))
            
        if len(trxs) == 0:
            print("{0:5.5} MINE REJECTED: SUBSTANDARD".format(clientId))
            return False

        print("{0:5.5} MINE APPROVED AND ACCEPTED: POQ".format(clientId))
        #print(block)
        self.newBlock(clientId, hashQ, trxs)
        return True


In [13]:
#classes_initial = [Node, Node, Node, Compounder, Node, OffChainNode, OffChainNode]
classes_initial = [Node, Node]
# At first, everyone is node | Assumed held global model
clients = setupClisF(train_inps, train_oups, test_inps, test_oups, rounds, [], classes_initial)
#binds = [[6,3],[5,3]]
#bindOCNodes(binds, clients)


In [14]:
blockchain = Blockchain()
# Assign blockchain
for c in clients:
    if c.__class__ != OffChainNode:
        c.assignBlockchain(blockchain)

list index out of range|Brand new blockchain, creating genesis block
BLK: C:0x20ec5#  0 <-- PH=0    :PF=0    , RL={}   , #TRXS=0
Blockchain created with ID: 0x20ec5102988 2262458968456


In [15]:
def combine_models_multiple(ms): # ms: list of models
    init_model = ms[0]
    for i in range(1, len(ms)):
        init_model[0] = np.add(init_model[0], init_model[i])
    return init_model

In [16]:
# for c in clients: train, send model to IPFS - get hash, send hash to blockchain as trx
# for m in miners: get trxs from blockchain backlog, 
def clientBatchRun(cli, epoch_, round_):
    # Train, send model to IPFS, get hash
    for r in range(round_):
        # Train and submit ===============================================
        for c in cli:
            if c.__class__ not in [Node, Compounder] or c.committee == False:
                continue
            loc = './models/'
            #c.train(loc, epoch_, r)
            c.train(loc, 1, r)
            c.submitModelAsTransaction()
            #c.blockchain.newTransaction(t) # bogus
        
        # Miners gather data ==============================================
        global_models = []
        for c in cli:
            global_model = c.aggregate(c.blockchain)
            if global_model != None:
                global_models.append(global_model)

        # COMPARE with oth miners
        # ==============================================
        # ProofOfQuality
        pass

        # REMOVE TRX FROM THOSE NOT IN FINAL C MINE BENCHMARKS (FILTER)
        #trx_new = []
        #for t in c.blockchain.current_transactions:
        #    if t.sender in sel_hashes:
        #        trx_new.append(t)

        print('round {} done'.format(round_))
        # if round == 1 ~ 2 || round % 5 == 0: all nodes mine, then assign best 2 as committee
            # check/compare: local PREV-NOW qual; global PREV-NOW qual; local vs global
            # substandard = drop transaction
            # aggregate old with new ones
                # miner check: which one is superior
                # top 2 promoted as committee
                # if top 2 was not committee, demote, promote new onws
        # if round > 2 && round != 5
            # miners mine like above
            # committee do not change
        break

In [17]:
clientBatchRun(clients[:2], epochs, rounds)

Cli# 0: 2.64 sec, Q: 0.6022; NDE:0 MOD-->CHIDX#  1
Cli# 1: 3.38 sec, Q: 0.5133; NDE:1 MOD-->CHIDX#  1
1     MINE APPROVED: POA
365/365 - 2s - loss: 0.0468 - accuracy: 0.6966
dfd5f 92.1567879499479 112.66665110063022
365/365 - 1s - loss: 0.0542 - accuracy: 0.6415
e2538 60.008920797549294 67.89753833148265
{}
1     MINE APPROVED: POA
183/183 - 1s - loss: 0.0466 - accuracy: 0.7036
dfd5f 93.07529039613233 112.66665110063022
183/183 - 1s - loss: 0.0542 - accuracy: 0.6441
e2538 60.24947034148879 67.89753833148265
{}
round 1 done
